In [1400]:
%create_livy_session --cluster bigdata-course-spark-cluster \
                     --id ses_test \
                     --conf spark.executor.instances=1 \
                     --conf spark.executor.cores=1

Execute error: Servant c1.4 not allocated: Allocation was interrupted

In [1450]:
%delete_livy_session  --cluster bigdata-course-spark-cluster-jdrtk8xm --id gr0373-Vlasyuk-Serebryakova

In [1451]:
%create_livy_session --cluster bigdata-course-spark-cluster-jdrtk8xm \
                     --id gr0373-Vlasyuk-Serebryakova \
                     --conf spark.executor.instances=1 \
                     --conf spark.executor.cores=4
#  #!spark —cluster bigdata-course-spark-cluster-11akvxnd —ses.sion gr0000-ivanov-petrov
#%delete_livy_session —cluster bigdata-course-spark-cluster-11akvxnd —id gr0000-ivanov-petrov

Starting session gr0373-Vlasyuk-Serebryakova


In [1452]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova
import numpy as np
import time
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import PowerIterationClustering
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from functools import reduce
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id
import random
from pyspark.mllib.evaluation import MulticlassMetrics

root=r"/user/bigdata-course/PAMAP2 Physical Activity Monitoring/PAMAP2_Dataset/Protocol/"

Waiting for an Apache Livy session to start...
Apache Livy session has started.



In [1453]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova
def load_activity_map():
    map = {}
    map[0] = 'transient'
    map[1] = 'lying'
    map[2] = 'sitting'
    map[3] = 'standing'
    map[4] = 'walking'
    map[5] = 'running'
    map[6] = 'cycling'
    map[7] = 'Nordic_walking'
    map[9] = 'watching_TV'
    map[10] = 'computer_work'
    map[11] = 'car driving'
    map[12] = 'ascending_stairs'
    map[13] = 'descending_stairs'
    map[16] = 'vacuum_cleaning'
    map[17] = 'ironing'
    map[18] = 'folding_laundry'
    map[19] = 'house_cleaning'
    map[20] = 'playing_soccer'
    map[24] = 'rope_jumping'
    return map

def generate_three_IMU(name):
    x = name +'_x'
    y = name +'_y'
    z = name +'_z'
    return [x,y,z]

def generate_four_IMU(name):
    x = name +'_x'
    y = name +'_y'
    z = name +'_z'
    w = name +'_w'
    return [x,y,z,w]

def generate_cols_IMU(name):
    # temp
    temp = name+'_temperature'
    output = [temp]
    # acceleration 16
    acceleration16 = name+'_3D_acc_16'
    acceleration16 = generate_three_IMU(acceleration16)
    output.extend(acceleration16)
    # acceleration 6
    acceleration6 = name+'_3D_acc_6'
    acceleration6 = generate_three_IMU(acceleration6)
    output.extend(acceleration6)
    # gyroscope
    gyroscope = name+'_3D_gyro'
    gyroscope = generate_three_IMU(gyroscope)
    output.extend(gyroscope)
    # magnometer
    magnometer = name+'_3D_magnetometer'
    magnometer = generate_three_IMU(magnometer)
    output.extend(magnometer)
    # oreintation
    oreintation = name+'_4D_orientation'
    oreintation = generate_four_IMU(oreintation)
    output.extend(oreintation)
    return output

def load_IMU():
    output = ['time_stamp','activity_id', 'heart_rate']
    hand = 'hand'
    hand = generate_cols_IMU(hand)
    output.extend(hand)
    chest = 'chest'
    chest = generate_cols_IMU(chest)
    output.extend(chest)
    ankle = 'ankle'
    ankle = generate_cols_IMU(ankle)
    output.extend(ankle)
    return output

def load_subjects(root=r"/user/bigdata-course/PAMAP2 Physical Activity Monitoring/PAMAP2_Dataset/Protocol/"):
    cols = load_IMU()
    
    path = root + "*.dat"
    rdd = sc.textFile(path)
    numbers_rdd = rdd.map(lambda x : (x.split(" "))).filter(lambda x: x[1] != '0')
    output = numbers_rdd.toDF(cols)
    return output

pamap_data = load_subjects()

for i in range(1, 10):
    #rdd = sc.textFile(path + f"subject{i}.dat")
    exec(f'subject_{i} = sc.textFile(root + f"subject10{i}.dat").map(lambda x : (x.split(" "))).filter(lambda x: x[1] != "0")')

import matplotlib.pyplot as plt
index = []
values = []
for i in range(1, 10):
    index.append(f"subject_{i}")
    values.append(globals()[index[-1]].count())

fig, ax = plt.subplots()
ax.bar(index, values)

plt.show()


## Обрабатывает датасет
Аппроксимируем Null и берем части датасета

In [1454]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova
def sum_line(line):  
    # Для каждой строки
    for i in range(number_of_feature):
        # Если значение не NaN, добавить его к сумме для данного столбца
        if(line[i]!='NaN'):
            sum_of_a_feature[i].add(float(line[i]))

def avg_column(accumulator,number_of_row):
    # Получить среднее значение для каждого столбца
    avg_feature=[]
    count=0
    for element in sum_of_a_feature:
        avg=element.value/number_of_row
        value=avg
        avg_feature.append(value)
        count+=1
    return avg_feature

def replace_missing_value(line,avg_feature):
    # Заменить отсутствующие значения на средние значения по столбцам
    for i in range(number_of_feature):
        if(line[i]=='NaN'):
            line[i]=float(avg_feature[i])
        else:
            line[i]=float(line[i])
    return line

def parsePoint(values):
    # Преобразовать строку в объект LabeledPoint, используя первый элемент в качестве метки
    return LabeledPoint(values[0], values[1:])
  
def process_subject(subject,number_of_feature,sum_of_a_feature):
    number_of_row=subject.count()
    # Вычислить сумму для каждого столбца
    subject.foreach(sum_line)
    # Вычислить среднее значение для каждого столбца
    avg_feature=avg_column(sum_of_a_feature,number_of_row)
    # Заменить отсутствующие значения на средние значения по столбцам
    subject_without_missing_value=subject.map(lambda j: replace_missing_value(j,avg_feature))
    # Вернуть преобразованный объект
    return subject_without_missing_value

def insert_label(x):
    # Вставить метку в начало списка значений
    result=x[1].tolist()
    result.insert(0,x[0])
    return result

number_of_row=subject_8.count()  # Подсчитываем количество строк в датасете subject_8
number_of_feature=54  # Устанавливаем количество признаков для датасета
sum_of_a_feature = [sc.accumulator(0) for x in range(number_of_feature)]  # Создаем список с аккумуляторами для каждого признака

subject_rdd_list = []  # Создаем пустой список для хранения RDD датасетов
for i in range(1, 9):
    #exec(f'subject{i} = process_subject(subject_{i},number_of_feature,sum_of_a_feature)')
    exec(f'subject{i} = process_subject(subject_{i},number_of_feature,sum_of_a_feature).map(lambda x:(((x[1],int(x[0]/5)),(np.asarray(x[2:]),1)))).reduceByKey(lambda a,b: (np.sum([a[0],b[0]], axis=0),a[1]+b[1])).map(lambda x: ((x[0][0]),np.true_divide(x[1][0],x[1][1]))).map(insert_label)')
    subject_rdd_list.append(globals()[f'subject{i}'])  # Добавляем каждый RDD датасета subject_i в список, используя глобальное имя переменной
    #rdd = sc.textbject{i} = process_subject(subject_{i},number_of_feature,sum_of_a_feature)')
   # exec(f'subject{i} = process_subject(subject_{i},number_of_feature,sum_of_a_feature).map(lambda x:(((x[1],int(x[0]/5)),(np.asarray(x[2:]),1)))).reduceByKey(lambda a,b: (np.sum([a[0],b[0]], axis=0),a[1]+b[1])).map(lambda x: ((x[0][0]),np.true_divide(x[1][0],x[1][1]))).map(insert_label)')
    


начинаем кластеризацию 

In [1444]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova
from pyspark.mllib.feature import StandardScaler
rdd = sc.union(subject_rdd_list)
scaler = StandardScaler(withMean=True, withStd=True)
scalerModel = scaler.fit(rdd)
normalizedData = scalerModel.transform(rdd)

print(scalerModel)

print(normalizedData)

MapPartitionsRDD[15018] at mapPartitions at PythonMLLibAPI.scala:1342


In [1455]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova
from pyspark.mllib.feature import StandardScaler
rdd = sc.union(subject_rdd_list)
scaler = StandardScaler(withMean=True, withStd=True)
scalerModel = scaler.fit(rdd)
normalizedData = scalerModel.transform(rdd)

#starting time
start = time.time()
clusters = KMeans.train(normalizedData, 3, maxIterations=10, initializationMode="random")
end = time.time()
# print("Cluster centers:")
# for center in clusters.centers:
#     print(center)
    
predictions = rdd.map(lambda x: clusters.predict(x))



pred_df = predictions.map(lambda x: (x, )).toDF(["cluster"])
pred_df = pred_df.withColumn("id", monotonically_increasing_id())

# Добавляем индекс к исходному DataFrame и объединяем по индексу
apr_df_id = pamap_data.withColumn("id", monotonically_increasing_id())
result_df = apr_df_id.join(pred_df, on="id", how="inner").drop("id")

# вывод количества записей в каждом кластере
result_df.groupBy("cluster").count().show()

# вывод типов активности в каждом кластере
result_df.groupBy("cluster", "activity_id").count().orderBy(["activity_id"]).show(20)

print("Execution time of the program is- ", end-start)

+-------+-----+
|cluster|count|
+-------+-----+
|      0|   69|
|      1| 1915|
|      2|  187|
+-------+-----+

+-------+-----------+-----+
|cluster|activity_id|count|
+-------+-----------+-----+
|      0|          1|   69|
|      2|          1|  165|
|      1|          1| 1657|
|      2|         24|   22|
|      1|         24|  258|
+-------+-----------+-----+

Execution time of the program is-  4.175935983657837


In [1456]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova

"""
WSSSE (Within Set Sum of Squared Error) - это метрика, используемая для оценки качества модели кластеризации. 
Она измеряет сумму квадратов расстояний между каждой точкой и центром кластера, к которому эта точка относится. 
Чем меньше WSSSE, тем лучше качество кластеризации.
"""
def error(point):
    center = clusters.centers[clusters.predict(point)]  # Находим центр кластера, к которому относится точка
    return sqrt(sum([x**2 for x in (point - center)]))  # Вычисляем расстояние между точкой и центром кластера

WSSSE = rdd.map(lambda point: error(point)).reduce(lambda x, y: x + y)  # Вычисляем сумму квадратов ошибок для каждой точки и суммируем их
print("Within Set Sum of Squared Error = " + str(WSSSE))  # Выводим сумму квадратов ошибок на экран

Within Set Sum of Squared Error = 462178.0808064495


In [1342]:
#!spark --cluster bigdata-course-spark-cluster-abq7rbdj --session gr0373-Vlasyuk-Serebryakova
# ne rabotaet :|
import numpy as np

# Вычисляем расстояние между точками и центроидами кластеров
def distance(point, center):
    return np.linalg.norm(point - center)

def calculate_distances(point, centers):
    return [distance(point, center) for center in centers]

# Вычисляем среднее расстояние до точек внутри кластера
def avg_distance_in_cluster(point, cluster_points):
    point = np.array(point)
    cluster_points = [np.array(p) for p in cluster_points]
    distances = [distance(point, p) for p in cluster_points if not np.array_equal(p, point)]
    return sum(distances) / len(distances) if distances else 0.0

def calculate_avg_distances(points, center, clusters):
    points = [np.array(p) for p in points]
    cluster_points = [p for p, c in zip(points, clusters) if c == center]
    return [avg_distance_in_cluster(point, cluster_points) for point in cluster_points]

# Вычисляем Silhouette Score
def silhouette_score(points, clusters):
    n = points.count()
    s = 0.0
    for point, cluster in zip(points.collect(), clusters.collect()):
        a = avg_distance_in_cluster(point, [p for p, c in zip(points.collect(), clusters.collect()) if c == cluster and not np.array_equal(p, point)])
        b = min(calculate_avg_distances(points.collect(), cluster, clusters.collect())) if len(set(clusters.collect())) > 1 else a
        s += (b - a) / max(a, b)
    return s / n

# Вычисляем Silhouette Score для модели KMeans
score = silhouette_score(rdd, clusters.predict(rdd))
print("Silhouette Score: ", score)


KeyboardInterrupt: 

In [1457]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova
import time

selected_columns = ["hand_3D_acc_6_x", "hand_3D_acc_6_y", "hand_3D_acc_6_z", 
                    "chest_3D_acc_6_x", "chest_3D_acc_6_y", "chest_3D_acc_6_z",
                    "ankle_3D_acc_6_x", "ankle_3D_acc_6_y", "ankle_3D_acc_6_z",
                    "hand_3D_acc_16_x", "hand_3D_acc_16_y", "hand_3D_acc_16_z",
                    "chest_3D_acc_16_x", "chest_3D_acc_16_y", "chest_3D_acc_16_z",
                    "ankle_3D_acc_16_x", "ankle_3D_acc_16_y", "ankle_3D_acc_16_z",
                    "hand_3D_gyro_x", "hand_3D_gyro_y", "hand_3D_gyro_z",
                    "hand_3D_magnetometer_x", "hand_3D_magnetometer_y", "hand_3D_magnetometer_z",
                    "hand_4D_orientation_w",
                    "chest_3D_gyro_x", "chest_3D_gyro_y", "chest_3D_gyro_z",
                    "chest_3D_magnetometer_x", "chest_3D_magnetometer_y", "chest_3D_magnetometer_z",
                    "chest_4D_orientation_w",
                    "ankle_3D_gyro_x", "ankle_3D_gyro_y", "ankle_3D_gyro_z",
                    "ankle_3D_magnetometer_x", "ankle_3D_magnetometer_y", "ankle_3D_magnetometer_z",
                    "ankle_4D_orientation_w", 
                   ]

from pyspark.sql.functions import col
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id

# for column in selected_columns:
#     pm_select = pamap_data.select(selected_columns).withColumn(column, col(column).cast("double"))
pamap_data_selected = pamap_data.select(
    [col(c).cast("double").alias(c) for c in selected_columns]
)

vecAssembler = VectorAssembler(inputCols=selected_columns, outputCol="features", handleInvalid="skip")
pm_select = vecAssembler.transform(pamap_data_selected)

rdd = pm_select.select("features").rdd.map(lambda x: Vectors.dense(x[0]))

scaler = StandardScaler(withMean=True, withStd=True)
scalerModel = scaler.fit(rdd)
normalizedData = scalerModel.transform(rdd)

from pyspark.mllib.clustering import KMeans
#starting time
start = time.time()
clusters = KMeans.train(rdd, 3, maxIterations=10, initializationMode="random")
end = time.time()

predictions = rdd.map(lambda x: clusters.predict(x))



pred_df = predictions.map(lambda x: (x, )).toDF(["cluster"])
pred_df = pred_df.withColumn("id", monotonically_increasing_id())

# Добавляем индекс к исходному DataFrame и объединяем по индексу
pamap_data = pamap_data.withColumn("id", monotonically_increasing_id())
result_df = pamap_data.join(pred_df, on="id", how="inner").drop("id")

# вывод количества записей в каждом кластере
result_df.groupBy("cluster").count().show()

# вывод типов активности в каждом кластере
result_df.groupBy("cluster", "activity_id").count().orderBy(["activity_id"]).show(50)

print("Execution time of the program is- ", end-start)


+-------+------+
|cluster| count|
+-------+------+
|      0|671966|
|      1|582659|
|      2|666806|
+-------+------+

+-------+-----------+------+
|cluster|activity_id| count|
+-------+-----------+------+
|      2|          1|  4520|
|      0|          1|  8684|
|      1|          1|179319|
|      2|         12| 38145|
|      1|         12| 10328|
|      0|         12| 68743|
|      2|         13| 40308|
|      0|         13| 42245|
|      1|         13| 22391|
|      2|         16| 40299|
|      0|         16|118166|
|      1|         16| 16888|
|      1|         17|107396|
|      0|         17| 21406|
|      2|         17|109888|
|      1|          2|147484|
|      0|          2|  5991|
|      2|          2| 31713|
|      0|         24|  4935|
|      1|         24|  2819|
|      2|         24| 28357|
|      2|          3|107863|
|      0|          3| 48418|
|      1|          3| 33650|
|      2|          4| 93981|
|      1|          4| 10217|
|      0|          4|133566|
|      0| 

In [1458]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = rdd.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 103128882.05271979


In [1424]:
#!spark --cluster bigdata-course-spark-cluster-abq7rbdj --session gr0373-Vlasyuk-Serebryakova

from pyspark.sql import SparkSession
from pyspark.mllib.clustering import GaussianMixture, GaussianMixtureModel
from pyspark.sql.functions import monotonically_increasing_id

def train_gmm(subject_rdd):
    # Создание экземпляра модели GMM
    gmm = GaussianMixture.train(subject_rdd, k=3, maxIterations=10)

    # Предсказание кластера для каждой точки в RDD
    predictions = subject_rdd.map(lambda x: gmm.predict(x))
    return predictions

# объединяем все rdd по кластерам
rdd = subject_rdd_list[0]
for subject_rdd in subject_rdd_list[1:]:
    rdd = rdd.union(subject_rdd)

# train GMM
predictions = train_gmm(rdd)

# Преобразование RDD в DataFrame
from pyspark.sql.functions import monotonically_increasing_id
pred_df = predictions.map(lambda x: (x, )).toDF(["cluster"])
pred_df = pred_df.withColumn("id", monotonically_increasing_id())

# Добавляем индекс к исходному DataFrame и объединяем по индексу
pamap_data = pamap_data.withColumn("id", monotonically_increasing_id())
result_df = pamap_data.join(pred_df, on="id", how="inner").drop("id")

# вывод количества записей в каждом кластере
result_df.groupBy("cluster").count().show()

# вывод типов активности в каждом кластере
result_df.groupBy("cluster", "activity_id").count().orderBy(["activity_id"]).show(20)



Execute error: Chosen cluster was not found.


# mirustal 

проба в классификацию потому что с кластеризацией трэээш

In [1463]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova
def load_activity_map():
    map = {}
    map[0] = 'transient'
    map[1] = 'lying'
    map[2] = 'sitting'
    map[3] = 'standing'
    map[4] = 'walking'
    map[5] = 'running'
    map[6] = 'cycling'
    map[7] = 'Nordic_walking'
    map[9] = 'watching_TV'
    map[10] = 'computer_work'
    map[11] = 'car driving'
    map[12] = 'ascending_stairs'
    map[13] = 'descending_stairs'
    map[16] = 'vacuum_cleaning'
    map[17] = 'ironing'
    map[18] = 'folding_laundry'
    map[19] = 'house_cleaning'
    map[20] = 'playing_soccer'
    map[24] = 'rope_jumping'
    return map

def generate_three_IMU(name):
    x = name +'_x'
    y = name +'_y'
    z = name +'_z'
    return [x,y,z]

def generate_four_IMU(name):
    x = name +'_x'
    y = name +'_y'
    z = name +'_z'
    w = name +'_w'
    return [x,y,z,w]

def generate_cols_IMU(name):
    # temp
    temp = name+'_temperature'
    output = [temp]
    # acceleration 16
    acceleration16 = name+'_3D_acc_16'
    acceleration16 = generate_three_IMU(acceleration16)
    output.extend(acceleration16)
    # acceleration 6
    acceleration6 = name+'_3D_acc_6'
    acceleration6 = generate_three_IMU(acceleration6)
    output.extend(acceleration6)
    # gyroscope
    gyroscope = name+'_3D_gyro'
    gyroscope = generate_three_IMU(gyroscope)
    output.extend(gyroscope)
    # magnometer
    magnometer = name+'_3D_magnetometer'
    magnometer = generate_three_IMU(magnometer)
    output.extend(magnometer)
    # oreintation
    oreintation = name+'_4D_orientation'
    oreintation = generate_four_IMU(oreintation)
    output.extend(oreintation)
    return output

def load_IMU():
    output = ['time_stamp','activity_id', 'heart_rate']
    hand = 'hand'
    hand = generate_cols_IMU(hand)
    output.extend(hand)
    chest = 'chest'
    chest = generate_cols_IMU(chest)
    output.extend(chest)
    ankle = 'ankle'
    ankle = generate_cols_IMU(ankle)
    output.extend(ankle)
    return output

def load_IMU_noTS():
    output = ['activity_id', 'heart_rate']
    hand = 'hand'
    hand = generate_cols_IMU(hand)
    output.extend(hand)
    chest = 'chest'
    chest = generate_cols_IMU(chest)
    output.extend(chest)
    ankle = 'ankle'
    ankle = generate_cols_IMU(ankle)
    output.extend(ankle)
    return output



def load_subjects(root=r"/user/bigdata-course/PAMAP2 Physical Activity Monitoring/PAMAP2_Dataset/Protocol/"):
    cols = load_IMU()
    
    path = root + "*.dat"
    rdd = sc.textFile(path)
    numbers_rdd = rdd.map(lambda x : (x.split(" "))).filter(lambda x: x[1] != '0')
    output = numbers_rdd.toDF(cols)
    return output

cluster_data = load_subjects()


for i in range(1, 10):
    #rdd = sc.textFile(path + f"subject{i}.dat")
    exec(f'subject_{i} = sc.textFile(root + f"subject10{i}.dat").map(lambda x : (x.split(" "))).filter(lambda x: x[1] != "0")')


index = []
values = []
for i in range(1, 10):
    index.append(f"subject_{i}")
    values.append(globals()[index[-1]].count())


import numpy as np

def sum_line(line):  
#For eache line 
    for i in range(number_of_feature):
        if(line[i]!='NaN'):
            sum_of_a_feature[i].add(float(line[i]))

def avg_column(accumulator,number_of_row):
#Get the average
    avg_feature=[]
    count=0
    for element in sum_of_a_feature:
        avg=element.value/number_of_row
        value=avg
        avg_feature.append(value)
        count+=1
    return avg_feature

def replace_missing_value(line,avg_feature):
    for i in range(number_of_feature):
        if(line[i]=='NaN'):
            line[i]=float(avg_feature[i])
        else:
            line[i]=float(line[i])
    return line

def parsePoint(values):
    values = list(map(float, values))
    return LabeledPoint(values[0], values[1:])
    return LabeledPoint(values[0], values[1:])
  
def process_subject(subject,number_of_feature,sum_of_a_feature):
    number_of_row=subject.count()
#     print(number_of_row)
#Compute sum for each column
    subject.foreach(sum_line)
#     print(subject.count())
#Compute avg of each column
    avg_feature=avg_column(sum_of_a_feature,number_of_row)
#     print(avg_feature)
#Replace missing value
    subject_without_missing_value=subject.map(lambda j: replace_missing_value(j,avg_feature))
#     print(subject_without_missing_value.count())
#     print(subject_without_missing_value)
#lambda j: processDataLine(j, arg1, arg2)
    return subject_without_missing_value

def insert_label(x):
    result=x[1].tolist()
    result.insert(0,x[0])
    return result

number_of_row=subject_8.count()
number_of_feature=54
sum_of_a_feature = [sc.accumulator(0) for x in range(number_of_feature)]
#print(subject_101.take(5))


subject_rdd_list = []
for i in range(1, 9):
    #rdd = sc.textFile(path + f"subject{i}.dat")
    exec(f'subject{i} = process_subject(subject_{i},number_of_feature,sum_of_a_feature).map(lambda x:(((x[1],int(x[0]/5)),(np.asarray(x[2:]),1)))).reduceByKey(lambda a,b: (np.sum([a[0],b[0]], axis=0),a[1]+b[1])).map(lambda x: ((x[0][0]),np.true_divide(x[1][0],x[1][1]))).map(insert_label)')
    #exec(f'subject{i} = process_subject(subject_{i},number_of_feature,sum_of_a_feature)')
    
    subject_rdd_list.append(globals()[f'subject{i}'])

clus_rdd = sc.union(subject_rdd_list)

In [1460]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova


list_error_rf=[]
list_error_lr=[]
list_error_dt=[]

time_rf = 0
time_dt = 0
time_nb = 0
time_lr = 0
for i in range(len(subject_rdd_list)):
    test_subject=subject_rdd_list[i]
    # Cоздаем новый список, который не включает в себя i-ый элемент
    training_subjects = subject_rdd_list[:i] + subject_rdd_list[i+1:]
    #Объединяем все RDD-объекты, кроме i-ого, в один RDD-объект
    dataset_subject_without_missing_value=sc.union(training_subjects)
    print(training_subjects)
    # Загружаем и парсим файл данных в RDD-объекты типа LabeledPoint

    data = dataset_subject_without_missing_value.map(parsePoint)
    # Разделяем данные на обучающую и тестовую выборки (30% данных оставляем на тестирование)
    trainingData = data # Все объекты, кроме i-го, используем для обучения
    testData = test_subject.map(parsePoint)  # Используем i-й объект для тестирования

    ################################################################# MODEL #############################################################################
    ###########Train a RandomForest model.######################
    '''
    Случайный лес (Random Forest) - это алгоритм машинного обучения, который используется
    для задач классификации, регрессии и кластеризации.
    Он состоит из множества решающих деревьев, которые обучаются на подвыборках данных
    и подмножествах признаков. Во время обучения каждое дерево строится независимо
    от других деревьев, и каждое дерево получает случайный поднабор данных. После обучения,
    классификация новых данных происходит путем применения каждого дерева к новым данным и выбора
    класса с наибольшим количеством голосов.
    
    
    
    numClasses: количество классов, на которые нужно разделить данные.
    categoricalFeaturesInfo: информация о категориальных признаках. Пустое значение означает, что все признаки являются непрерывными.
    numTrees: количество деревьев в лесу. В данном случае равно 2000.
    featureSubsetStrategy: стратегия выбора подмножества признаков для каждого дерева. Установлено значение "auto", которое позволяет алгоритму выбрать подходящую стратегию.
    impurity: критерий для оценки качества разделения узлов в деревьях. Установлено значение 'gini'.
    maxDepth: максимальная глубина деревьев. Установлено значение 5.
    maxBins: максимальное количество корзин для дискретизации признаков. Установлено значение 32.
    '''
    timeRF_start = time.time()
    model = RandomForest.trainClassifier(trainingData, numClasses=25, categoricalFeaturesInfo={},
                                       numTrees=2000, featureSubsetStrategy="auto",
                                       impurity='gini', maxDepth=5, maxBins=32)
    timeRF_end = time.time()
    time_rf += timeRF_end - timeRF_start
    # Оценка модели на тестовых данных и вычисление ошибки тестирования
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    
    testErr = labelsAndPredictions.filter(lambda vp: vp[0] != vp[1]).count()  / float(testData.count())
    list_error_rf.append(testErr) 
    #Save the error on the leave one out 
    '''
    "Leave one out" (LOO) - это метод кросс-валидации, используемый для оценки производительности
    моделей машинного обучения. Он заключается в том, что одно наблюдение удаляется из обучающего
    набора данных, модель обучается на оставшихся данных и затем используется для прогнозирования
    значения удаленного наблюдения. Этот процесс повторяется для всех наблюдений в обучающем наборе,
    и каждый раз ошибка прогнозирования рассчитывается для удаленного наблюдения.
    list_error_rf.append(testErr) 
    '''
    #print('Test Error = ' + str(testErr))
    ########### Logistic Regression model.#######
    '''
    Logistic regression (логистическая регрессия) - это метод машинного обучения, который применяется
    для задач классификации. Он использует логистическую функцию, чтобы преобразовать
    линейную комбинацию входных признаков в вероятность отнесения объекта к определенному классу.

    В данном коде происходит обучение модели логистической регрессии с помощью метода оптимизации LBFGS
    (Limited-memory Broyden–Fletcher–Goldfarb–Shanno).
    Этот метод используется для решения задач оптимизации без ограничений с помощью градиентного спуска.

    Параметры:
    trainingData - обучающая выборка;
    numClasses - число классов в задаче классификации;
    '''
    timeLR_start = time.time()
    model_regression = LogisticRegressionWithLBFGS.train(trainingData,numClasses=25)
    timeLR_end = time.time()
    time_lr += timeLR_end - timeLR_start
    # Оценка модели на тестовых данных
    labelsAndPreds = testData.map(lambda p: (p.label, model_regression.predict(p.features)))
    trainErr = labelsAndPreds.filter(lambda vp: vp[0] != vp[1]).count() / float(data.count())
    # Сохранение ошибки на leave one out
    list_error_lr.append(testErr)
    #print("Training Error = " + str(trainErr))
    
    ########### Decision Tree .######################
    '''
    Decision Tree (дерево решений) - это один из самых популярных алгоритмов машинного обучения,
    который используется как для задач классификации, так и для задач регрессии.
    Он представляет собой структуру дерева, где каждый узел представляет собой тест на определенный
    признак, каждое ребро соответствует значению этого признака,
    а каждый лист соответствует конкретному классу или значению целевой переменной.
    
    Параметры:

    numClasses: число уникальных классов в целевой переменной.
    categoricalFeaturesInfo: словарь, который содержит информацию о категориальных признаках. По умолчанию все признаки считаются непрерывными.
    impurity: функция, которая используется для вычисления неопределенности узла. Возможные значения - "gini" для индекса Джини и "entropy" для энтропии.
    maxDepth: максимальная глубина дерева. Более глубокое дерево может привести к переобучению, тогда как более неглубокое дерево может привести к недообучению.
    maxBins: максимальное количество корзин, используемых для дискретизации непрерывных признаков.
    '''
    
    
    #Пустые categoricalFeaturesInfo указывают на то, что все признаки являются непрерывными.
    timeDT_start = time.time()
    model = DecisionTree.trainClassifier(trainingData, numClasses=25, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)
    timeDT_end = time.time()
    time_dt += timeDT_end - timeDT_start
    # Evaluate model on test instances and compute test error
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    testErr = labelsAndPredictions.filter(lambda vp: vp[0] != vp[1]).count() / float(testData.count())
   # Оценка модели на тестовых данных и вычисление ошибки
    list_error_dt.append(testErr)
    #print('Test Error = ' + str(testErr))
    #print(model.toDebugString())
    
    
########### Naives Bayes .######################
# Список RDD, содержащих данные по различным предметам
subject_rdd_list=[subject1,subject2,subject3,subject4,subject5,subject6, subject7, subject8]
list_error_nb=[]
# timeNB_start = []
# timeNB_end = []
for i in range(len(subject_rdd_list)):
    subject_rdd_list=[subject1,subject2,subject3,subject4,subject5,subject6, subject7, subject8]
    # Извлечение тестовых данных (одного предмета)
    test_subject=subject_rdd_list[i].map(lambda x: (map(abs, x)))
    del subject_rdd_list[i] # Исключение тестовых данных из списка RDD
    #Объединение оставшихся RDD, чтобы сформировать обучающий набор данных
    dataset_subject_without_missing_value=sc.union(subject_rdd_list)
    # Загрузка и парсинг файлов с данными в RDD объект LabeledPoint
    data = dataset_subject_without_missing_value.map(lambda x: (map(abs, x))).map(parsePoint)
    trainingData = data 
    testData = test_subject.map(parsePoint)
    ########## Naive Bayes model.######################
    '''
    Наивный Байесовский классификатор - это вероятностный алгоритм машинного обучения,
    основанный на теореме Байеса. Он используется для классификации объектов на основе комбинации
    априорной вероятности и правдоподобия условия.

    В машинном обучении наивный Байесовский классификатор является простым и эффективным алгоритмом,
    особенно для категориальных и текстовых данных. Он основывается на предположении о независимости
    между признаками, то есть каждый признак вносит независимый вклад в прогноз классификации объекта.

    lambda_: Параметр сглаживания Лапласа, который используется для предотвращения вероятности,
    равной нулю.
    '''
    timeNB_start = time.time()
    model = NaiveBayes.train(trainingData, 1.0)
    timeNB_end = time.time()
    time_nb += timeNB_end - timeNB_start
     # Прогнозирование и проверка точности
    predictionAndLabel = testData.map(lambda p: (model.predict(p.features), p.label))
    accuracy = 1.0 * predictionAndLabel.filter(lambda vp: vp[0] != vp[1]).count() / testData.count()
    list_error_nb.append(accuracy)

print("Time Random Forest ", time_rf)
print("Average Error Random Forest ")
print(reduce(lambda x, y: x + y, list_error_rf) / len(list_error_rf))


print("Time LogisticRegression ", time_lr)
print("Average Error Logistic Regression")
print(reduce(lambda x, y: x + y, list_error_lr) / len(list_error_lr))


print("Time DecisionTree ", time_dt)
print("Average Error Decision Tree")
print(reduce(lambda x, y: x + y, list_error_dt) / len(list_error_dt))


print("Time Naive Bayes ", time_nb)
print("Average Error Naive Bayes")
print(reduce(lambda x, y: x + y, list_error_nb) / len(list_error_nb))


print("RF: ")
#####################################      RF

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint(row[0], row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint(row[0], row[1:]))
time_s = time.time()
model = RandomForest.trainClassifier(train_rdd, numClasses=25, categoricalFeaturesInfo={},
                                       numTrees=2000, featureSubsetStrategy="auto",
                                       impurity='gini', maxDepth=5, maxBins=32)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)
matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time Random Forest ",  time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())



metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

print("LR:")
####################################  LR


test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))



time_s = time.time()
model = LogisticRegressionWithLBFGS.train(train_rdd,numClasses=25)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())

labels_and_predictions = test_rdd.map(lambda lp: (lp.label, float(model.predict(lp.features))))


metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)


print("DT:")
########################################################### DT

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))



time_s = time.time()
model = DecisionTree.trainClassifier(train_rdd, numClasses=25, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())



metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

print("NB:")
####################################### NB

# Вычисление метрик качества модели

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda x: (map(abs, x))).map(parsePoint)
test_rdd = test_df.rdd.map(lambda x: (map(abs, x))).map(parsePoint)



time_s = time.time()
model = NaiveBayes.train(train_rdd, 1.0)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())
labels_and_predictions = test_rdd.map(lambda lp: (lp.label, float(model.predict(lp.features))))

metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

    

[PythonRDD[326] at RDD at PythonRDD.scala:53, PythonRDD[327] at RDD at PythonRDD.scala:53, PythonRDD[328] at RDD at PythonRDD.scala:53, PythonRDD[329] at RDD at PythonRDD.scala:53, PythonRDD[330] at RDD at PythonRDD.scala:53, PythonRDD[331] at RDD at PythonRDD.scala:53, PythonRDD[332] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, PythonRDD[327] at RDD at PythonRDD.scala:53, PythonRDD[328] at RDD at PythonRDD.scala:53, PythonRDD[329] at RDD at PythonRDD.scala:53, PythonRDD[330] at RDD at PythonRDD.scala:53, PythonRDD[331] at RDD at PythonRDD.scala:53, PythonRDD[332] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, PythonRDD[326] at RDD at PythonRDD.scala:53, PythonRDD[328] at RDD at PythonRDD.scala:53, PythonRDD[329] at RDD at PythonRDD.scala:53, PythonRDD[330] at RDD at PythonRDD.scala:53, PythonRDD[331] at RDD at PythonRDD.scala:53, PythonRDD[332] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, Python

In [1461]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova


list_error_rf=[]
list_error_lr=[]
list_error_dt=[]
subject_rdd_list = subject_rdd_list[0:6]

time_rf = 0
time_dt = 0
time_nb = 0
time_lr = 0
for i in range(len(subject_rdd_list)):
    test_subject=subject_rdd_list[i]
    # Cоздаем новый список, который не включает в себя i-ый элемент
    training_subjects = subject_rdd_list[:i] + subject_rdd_list[i+1:]
    #Объединяем все RDD-объекты, кроме i-ого, в один RDD-объект
    dataset_subject_without_missing_value=sc.union(training_subjects)
    print(training_subjects)
    # Загружаем и парсим файл данных в RDD-объекты типа LabeledPoint

    data = dataset_subject_without_missing_value.map(parsePoint)
    # Разделяем данные на обучающую и тестовую выборки (30% данных оставляем на тестирование)
    trainingData = data # Все объекты, кроме i-го, используем для обучения
    testData = test_subject.map(parsePoint)  # Используем i-й объект для тестирования

    ################################################################# MODEL #############################################################################
    ###########Train a RandomForest model.######################
    '''
    Случайный лес (Random Forest) - это алгоритм машинного обучения, который используется
    для задач классификации, регрессии и кластеризации.
    Он состоит из множества решающих деревьев, которые обучаются на подвыборках данных
    и подмножествах признаков. Во время обучения каждое дерево строится независимо
    от других деревьев, и каждое дерево получает случайный поднабор данных. После обучения,
    классификация новых данных происходит путем применения каждого дерева к новым данным и выбора
    класса с наибольшим количеством голосов.
    
    
    
    numClasses: количество классов, на которые нужно разделить данные.
    categoricalFeaturesInfo: информация о категориальных признаках. Пустое значение означает, что все признаки являются непрерывными.
    numTrees: количество деревьев в лесу. В данном случае равно 2000.
    featureSubsetStrategy: стратегия выбора подмножества признаков для каждого дерева. Установлено значение "auto", которое позволяет алгоритму выбрать подходящую стратегию.
    impurity: критерий для оценки качества разделения узлов в деревьях. Установлено значение 'gini'.
    maxDepth: максимальная глубина деревьев. Установлено значение 5.
    maxBins: максимальное количество корзин для дискретизации признаков. Установлено значение 32.
    '''
    timeRF_start = time.time()
    model = RandomForest.trainClassifier(trainingData, numClasses=25, categoricalFeaturesInfo={},
                                       numTrees=2000, featureSubsetStrategy="auto",
                                       impurity='gini', maxDepth=5, maxBins=32)
    timeRF_end = time.time()
    time_rf += timeRF_end - timeRF_start
    # Оценка модели на тестовых данных и вычисление ошибки тестирования
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    
    testErr = labelsAndPredictions.filter(lambda vp: vp[0] != vp[1]).count()  / float(testData.count())
    list_error_rf.append(testErr) 
    #Save the error on the leave one out 
    '''
    "Leave one out" (LOO) - это метод кросс-валидации, используемый для оценки производительности
    моделей машинного обучения. Он заключается в том, что одно наблюдение удаляется из обучающего
    набора данных, модель обучается на оставшихся данных и затем используется для прогнозирования
    значения удаленного наблюдения. Этот процесс повторяется для всех наблюдений в обучающем наборе,
    и каждый раз ошибка прогнозирования рассчитывается для удаленного наблюдения.
    list_error_rf.append(testErr) 
    '''
    #print('Test Error = ' + str(testErr))
    ########### Logistic Regression model.#######
    '''
    Logistic regression (логистическая регрессия) - это метод машинного обучения, который применяется
    для задач классификации. Он использует логистическую функцию, чтобы преобразовать
    линейную комбинацию входных признакоов в вероятность отнесения объекта к определенному классу.

    В данном коде происходит обучение модели логистической регрессии с помощью метода оптимизации LBFGS
    (Limited-memory Broyden–Fletcher–Goldfarb–Shanno).
    Этот метод используется для решения задач оптимизации без ограничений с помощью градиентного спуска.

    Параметры:
    trainingData - обучающая выборка;
    numClasses - число классов в задаче классификации;
    '''
    timeLR_start = time.time()
    model_regression = LogisticRegressionWithLBFGS.train(trainingData,numClasses=25)
    timeLR_end = time.time()
    time_lr += timeLR_end - timeLR_start
    # Оценка модели на тестовых данных
    labelsAndPreds = testData.map(lambda p: (p.label, model_regression.predict(p.features)))
    trainErr = labelsAndPreds.filter(lambda vp: vp[0] != vp[1]).count() / float(data.count())
    # Сохранение ошибки на leave one out
    list_error_lr.append(testErr)
    #print("Training Error = " + str(trainErr))
    
    ########### Decision Tree .######################
    '''
    Decision Tree (дерево решений) - это один из самых популярных алгоритмов машинного обучения,
    который используется как для задач классификации, так и для задач регрессии.
    Он представляет собой структуру дерева, где каждый узел представляет собой тест на определенный
    признак, каждое ребро соответствует значению этого признака,
    а каждый лист соответствует конкретному классу или значению целевой переменной.
    
    Параметры:

    numClasses: число уникальных классов в целевой переменной.
    categoricalFeaturesInfo: словарь, который содержит информацию о категориальных признаках. По умолчанию все признаки считаются непрерывными.
    impurity: функция, которая используется для вычисления неопределенности узла. Возможные значения - "gini" для индекса Джини и "entropy" для энтропии.
    maxDepth: максимальная глубина дерева. Более глубокое дерево может привести к переобучению, тогда как более неглубокое дерево может привести к недообучению.
    maxBins: максимальное количество корзин, используемых для дискретизации непрерывных признаков.
    '''
    
    
    #Пустые categoricalFeaturesInfo указывают на то, что все признаки являются непрерывными.
    timeDT_start = time.time()
    model = DecisionTree.trainClassifier(trainingData, numClasses=25, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)
    timeDT_end = time.time()
    time_dt += timeDT_end - timeDT_start
    # Evaluate model on test instances and compute test error
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    testErr = labelsAndPredictions.filter(lambda vp: vp[0] != vp[1]).count() / float(testData.count())
   # Оценка модели на тестовых данных и вычисление ошибки
    list_error_dt.append(testErr)
    #print('Test Error = ' + str(testErr))
    #print(model.toDebugString())
    
    
########### Naives Bayes .######################
# Список RDD, содержащих данные по различным предметам
subject_rdd_list=[subject1,subject2,subject3,subject4,subject5,subject6, subject7, subject8]
list_error_nb=[]
# timeNB_start = []
# timeNB_end = []
for i in range(len(subject_rdd_list)):
    subject_rdd_list=[subject1,subject2,subject3,subject4,subject5,subject6, subject7, subject8]
    # Извлечение тестовых данных (одного предмета)
    test_subject=subject_rdd_list[i].map(lambda x: (map(abs, x)))
    del subject_rdd_list[i] # Исключение тестовых данных из списка RDD
    #Объединение оставшихся RDD, чтобы сформировать обучающий набор данных
    dataset_subject_without_missing_value=sc.union(subject_rdd_list)
    # Загрузка и парсинг файлов с данными в RDD объект LabeledPoint
    data = dataset_subject_without_missing_value.map(lambda x: (map(abs, x))).map(parsePoint)
    trainingData = data 
    testData = test_subject.map(parsePoint)
    ########## Naive Bayes model.######################
    '''
    Наивный Байесовский классификатор - это вероятностный алгоритм машинного обучения,
    основанный на теореме Байеса. Он используется для классификации объектов на основе комбинации
    априорной вероятности и правдоподобия условия.

    В машинном обучении наивный Байесовский классификатор является простым и эффективным алгоритмом,
    особенно для категориальных и текстовых данных. Он основывается на предположении о независимости
    между признаками, то есть каждый признак вносит независимый вклад в прогноз классификации объекта.

    lambda_: Параметр сглаживания Лапласа, который используется для предотвращения вероятности,
    равной нулю.
    '''
    timeNB_start = time.time()
    model = NaiveBayes.train(trainingData, 1.0)
    timeNB_end = time.time()
    time_nb += timeNB_end - timeNB_start
     # Прогнозирование и проверка точности
    predictionAndLabel = testData.map(lambda p: (model.predict(p.features), p.label))
    accuracy = 1.0 * predictionAndLabel.filter(lambda vp: vp[0] != vp[1]).count() / testData.count()
    list_error_nb.append(accuracy)

print("Time Random Forest ", time_rf)
print("Average Error Random Forest ")
print(reduce(lambda x, y: x + y, list_error_rf) / len(list_error_rf))


print("Time LogisticRegression ", time_lr)
print("Average Error Logistic Regression")
print(reduce(lambda x, y: x + y, list_error_lr) / len(list_error_lr))


print("Time DecisionTree ", time_dt)
print("Average Error Decision Tree")
print(reduce(lambda x, y: x + y, list_error_dt) / len(list_error_dt))


print("Time Naive Bayes ", time_nb)
print("Average Error Naive Bayes")
print(reduce(lambda x, y: x + y, list_error_nb) / len(list_error_nb))


print("RF: ")
#####################################      RF

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint(row[0], row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint(row[0], row[1:]))
time_s = time.time()
model = RandomForest.trainClassifier(train_rdd, numClasses=25, categoricalFeaturesInfo={},
                                       numTrees=2000, featureSubsetStrategy="auto",
                                       impurity='gini', maxDepth=5, maxBins=32)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)
matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time Random Forest ",  time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())



metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

print("LR:")
####################################  LR


test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))



time_s = time.time()
model = LogisticRegressionWithLBFGS.train(train_rdd,numClasses=25)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())

labels_and_predictions = test_rdd.map(lambda lp: (lp.label, float(model.predict(lp.features))))


metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)


print("DT:")
########################################################### DT

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))



time_s = time.time()
model = DecisionTree.trainClassifier(train_rdd, numClasses=25, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())



metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

print("NB:")
####################################### NB

# Вычисление метрик качества модели

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda x: (map(abs, x))).map(parsePoint)
test_rdd = test_df.rdd.map(lambda x: (map(abs, x))).map(parsePoint)



time_s = time.time()
model = NaiveBayes.train(train_rdd, 1.0)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())
labels_and_predictions = test_rdd.map(lambda lp: (lp.label, float(model.predict(lp.features))))

metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

    

[PythonRDD[326] at RDD at PythonRDD.scala:53, PythonRDD[327] at RDD at PythonRDD.scala:53, PythonRDD[328] at RDD at PythonRDD.scala:53, PythonRDD[329] at RDD at PythonRDD.scala:53, PythonRDD[330] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, PythonRDD[327] at RDD at PythonRDD.scala:53, PythonRDD[328] at RDD at PythonRDD.scala:53, PythonRDD[329] at RDD at PythonRDD.scala:53, PythonRDD[330] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, PythonRDD[326] at RDD at PythonRDD.scala:53, PythonRDD[328] at RDD at PythonRDD.scala:53, PythonRDD[329] at RDD at PythonRDD.scala:53, PythonRDD[330] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, PythonRDD[326] at RDD at PythonRDD.scala:53, PythonRDD[327] at RDD at PythonRDD.scala:53, PythonRDD[329] at RDD at PythonRDD.scala:53, PythonRDD[330] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, PythonRDD[326] at RDD at PythonRDD.scala:53, Pytho

In [1462]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova


list_error_rf=[]
list_error_lr=[]
list_error_dt=[]
subject_rdd_list = subject_rdd_list[0:3]

time_rf = 0
time_dt = 0
time_nb = 0
time_lr = 0
for i in range(len(subject_rdd_list)):
    test_subject=subject_rdd_list[i]
    # Cоздаем новый список, который не включает в себя i-ый элемент
    training_subjects = subject_rdd_list[:i] + subject_rdd_list[i+1:]
    #Объединяем все RDD-объекты, кроме i-ого, в один RDD-объект
    dataset_subject_without_missing_value=sc.union(training_subjects)
    print(training_subjects)
    # Загружаем и парсим файл данных в RDD-объекты типа LabeledPoint

    data = dataset_subject_without_missing_value.map(parsePoint)
    # Разделяем данные на обучающую и тестовую выборки (30% данных оставляем на тестирование)
    trainingData = data # Все объекты, кроме i-го, используем для обучения
    testData = test_subject.map(parsePoint)  # Используем i-й объект для тестирования

    ################################################################# MODEL #############################################################################
    ###########Train a RandomForest model.######################
    '''
    Случайный лес (Random Forest) - это алгоритм машинного обучения, который используется
    для задач классификации, регрессии и кластеризации.
    Он состоит из множества решающих деревьев, которые обучаются на подвыборках данных
    и подмножествах признаков. Во время обучения каждое дерево строится независимо
    от других деревьев, и каждое дерево получает случайный поднабор данных. После обучения,
    классификация новых данных происходит путем применения каждого дерева к новым данным и выбора
    класса с наибольшим количеством голосов.
    
    
    
    numClasses: количество классов, на которые нужно разделить данные.
    categoricalFeaturesInfo: информация о категориальных признаках. Пустое значение означает, что все признаки являются непрерывными.
    numTrees: количество деревьев в лесу. В данном случае равно 2000.
    featureSubsetStrategy: стратегия выбора подмножества признаков для каждого дерева. Установлено значение "auto", которое позволяет алгоритму выбрать подходящую стратегию.
    impurity: критерий для оценки качества разделения узлов в деревьях. Установлено значение 'gini'.
    maxDepth: максимальная глубина деревьев. Установлено значение 5.
    maxBins: максимальное количество корзин для дискретизации признаков. Установлено значение 32.
    '''
    timeRF_start = time.time()
    model = RandomForest.trainClassifier(trainingData, numClasses=25, categoricalFeaturesInfo={},
                                       numTrees=2000, featureSubsetStrategy="auto",
                                       impurity='gini', maxDepth=5, maxBins=32)
    timeRF_end = time.time()
    time_rf += timeRF_end - timeRF_start
    # Оценка модели на тестовых данных и вычисление ошибки тестирования
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    
    testErr = labelsAndPredictions.filter(lambda vp: vp[0] != vp[1]).count()  / float(testData.count())
    list_error_rf.append(testErr) 
    #Save the error on the leave one out 
    '''
    "Leave one out" (LOO) - это метод кросс-валидации, используемый для оценки производительности
    моделей машинного обучения. Он заключается в том, что одно наблюдение удаляется из обучающего
    набора данных, модель обучается на оставшихся данных и затем используется для прогнозирования
    значения удаленного наблюдения. Этот процесс повторяется для всех наблюдений в обучающем наборе,
    и каждый раз ошибка прогнозирования рассчитывается для удаленного наблюдения.
    list_error_rf.append(testErr) 
    '''
    #print('Test Error = ' + str(testErr))
    ########### Logistic Regression model.#######
    '''
    Logistic regression (логистическая регрессия) - это метод машинного обучения, который применяется
    для задач классификации. Он использует логистическую функцию, чтобы преобразовать
    линейную комбинацию входных признаков в вероятность отнесения объекта к определенному классу.

    В данном коде происходит обучение модели логистической регрессии с помощью метода оптимизации LBFGS
    (Limited-memory Broyden–Fletcher–Goldfarb–Shanno).
    Этот метод используется для решения задач оптимизации без ограничений с помощью градиентного спуска.

    Параметры:
    trainingData - обучающая выборка;
    numClasses - число классов в задаче классификации;
    '''
    timeLR_start = time.time()
    model_regression = LogisticRegressionWithLBFGS.train(trainingData,numClasses=25)
    timeLR_end = time.time()
    time_lr += timeLR_end - timeLR_start
    # Оценка модели на тестовых данных
    labelsAndPreds = testData.map(lambda p: (p.label, model_regression.predict(p.features)))
    trainErr = labelsAndPreds.filter(lambda vp: vp[0] != vp[1]).count() / float(data.count())
    # Сохранение ошибки на leave one out
    list_error_lr.append(testErr)
    #print("Training Error = " + str(trainErr))
    
    ########### Decision Tree .######################
    '''
    Decision Tree (дерево решений) - это один из самых популярных алгоритмов машинного обучения,
    который используется как для задач классификации, так и для задач регрессии.
    Он представляет собой структуру дерева, где каждый узел представляет собой тест на определенный
    признак, каждое ребро соответствует значению этого признака,
    а каждый лист соответствует конкретному классу или значению целевой переменной.
    
    Параметры:

    numClasses: число уникальных классов в целевой переменной.
    categoricalFeaturesInfo: словарь, который содержит информацию о категориальных признаках. По умолчанию все признаки считаются непрерывными.
    impurity: функция, которая используется для вычисления неопределенности узла. Возможные значения - "gini" для индекса Джини и "entropy" для энтропии.
    maxDepth: максимальная глубина дерева. Более глубокое дерево может привести к переобучению, тогда как более неглубокое дерево может привести к недообучению.
    maxBins: максимальное количество корзин, используемых для дискретизации непрерывных признаков.
    '''
    
    
    #Пустые categoricalFeaturesInfo указывают на то, что все признаки являются непрерывными.
    timeDT_start = time.time()
    model = DecisionTree.trainClassifier(trainingData, numClasses=25, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)
    timeDT_end = time.time()
    time_dt += timeDT_end - timeDT_start
    # Evaluate model on test instances and compute test error
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    testErr = labelsAndPredictions.filter(lambda vp: vp[0] != vp[1]).count() / float(testData.count())
   # Оценка модели на тестовых данных и вычисление ошибки
    list_error_dt.append(testErr)
    #print('Test Error = ' + str(testErr))
    #print(model.toDebugString())
    
    
########### Naives Bayes .######################
# Список RDD, содержащих данные по различным предметам
subject_rdd_list=[subject1,subject2,subject3,subject4,subject5,subject6, subject7, subject8]
list_error_nb=[]
# timeNB_start = []
# timeNB_end = []
for i in range(len(subject_rdd_list)):
    subject_rdd_list=[subject1,subject2,subject3,subject4,subject5,subject6, subject7, subject8]
    # Извлечение тестовых данных (одного предмета)
    test_subject=subject_rdd_list[i].map(lambda x: (map(abs, x)))
    del subject_rdd_list[i] # Исключение тестовых данных из списка RDD
    #Объединение оставшихся RDD, чтобы сформировать обучающий набор данных
    dataset_subject_without_missing_value=sc.union(subject_rdd_list)
    # Загрузка и парсинг файлов с данными в RDD объект LabeledPoint
    data = dataset_subject_without_missing_value.map(lambda x: (map(abs, x))).map(parsePoint)
    trainingData = data 
    testData = test_subject.map(parsePoint)
    ########## Naive Bayes model.######################
    '''
    Наивный Байесовский классификатор - это вероятностный алгоритм машинного обучения,
    основанный на теореме Байеса. Он используется для классификации объектов на основе комбинации
    априорной вероятности и правдоподобия условия.

    В машинном обучении наивный Байесовский классификатор является простым и эффективным алгоритмом,
    особенно для категориальных и текстовых данных. Он основывается на предположении о независимости
    между признаками, то есть каждый признак вносит независимый вклад в прогноз классификации объекта.

    lambda_: Параметр сглаживания Лапласа, который используется для предотвращения вероятности,
    равной нулю.
    '''
    timeNB_start = time.time()
    model = NaiveBayes.train(trainingData, 1.0)
    timeNB_end = time.time()
    time_nb += timeNB_end - timeNB_start
     # Прогнозирование и проверка точности
    predictionAndLabel = testData.map(lambda p: (model.predict(p.features), p.label))
    accuracy = 1.0 * predictionAndLabel.filter(lambda vp: vp[0] != vp[1]).count() / testData.count()
    list_error_nb.append(accuracy)

print("Time Random Forest ", time_rf)
print("Average Error Random Forest ")
print(reduce(lambda x, y: x + y, list_error_rf) / len(list_error_rf))


print("Time LogisticRegression ", time_lr)
print("Average Error Logistic Regression")
print(reduce(lambda x, y: x + y, list_error_lr) / len(list_error_lr))


print("Time DecisionTree ", time_dt)
print("Average Error Decision Tree")
print(reduce(lambda x, y: x + y, list_error_dt) / len(list_error_dt))


print("Time Naive Bayes ", time_nb)
print("Average Error Naive Bayes")
print(reduce(lambda x, y: x + y, list_error_nb) / len(list_error_nb))


print("RF: ")
#####################################      RF

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint(row[0], row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint(row[0], row[1:]))
time_s = time.time()
model = RandomForest.trainClassifier(train_rdd, numClasses=25, categoricalFeaturesInfo={},
                                       numTrees=2000, featureSubsetStrategy="auto",
                                       impurity='gini', maxDepth=5, maxBins=32)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)
matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time Random Forest ",  time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())



metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

print("LR:")
####################################  LR


test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))



time_s = time.time()
model = LogisticRegressionWithLBFGS.train(train_rdd,numClasses=25)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())

labels_and_predictions = test_rdd.map(lambda lp: (lp.label, float(model.predict(lp.features))))


metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)


print("DT:")
########################################################### DT

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))
test_rdd = test_df.rdd.map(lambda row : LabeledPoint((row[0]), row[1:]))



time_s = time.time()
model = DecisionTree.trainClassifier(train_rdd, numClasses=25, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())



metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

print("NB:")
####################################### NB

# Вычисление метрик качества модели

test_subject_df = sc.union(subject_rdd_list).toDF(load_IMU_noTS())

train_df, test_df = test_subject_df.randomSplit([0.8, 0.2])

train_rdd = train_df.rdd.map(lambda x: (map(abs, x))).map(parsePoint)
test_rdd = test_df.rdd.map(lambda x: (map(abs, x))).map(parsePoint)



time_s = time.time()
model = NaiveBayes.train(train_rdd, 1.0)
time_e = time.time()

predictions = model.predict(test_rdd.map(lambda x: x.features))
labels_and_predictions = test_rdd.map(lambda lp: lp.label).zip(predictions)

matched_predictions_cont = labels_and_predictions.filter(lambda lp: lp[0] == lp[1]).count()
accuracy_percent = matched_predictions_cont * 100 / float(test_rdd.count())
print("Доля верных предсказаний:", str(accuracy_percent) + "%")
print("Time LogisticRegression ", time_e - time_s)
#print("Структура обученной модели:")
#print(model.toDebugString())
labels_and_predictions = test_rdd.map(lambda lp: (lp.label, float(model.predict(lp.features))))

metrics = None
metrics = MulticlassMetrics(labels_and_predictions)

# Вычисление метрик качества модели

precision = metrics.weightedPrecision
recall = metrics.weightedRecall
f1Score = metrics.weightedFMeasure()
accuracy = metrics.accuracy

# Вывод метрик
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1-score = %s" % f1Score)
print("accuracy: = %s" % accuracy)

    

[PythonRDD[326] at RDD at PythonRDD.scala:53, PythonRDD[327] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, PythonRDD[327] at RDD at PythonRDD.scala:53]
[PythonRDD[325] at RDD at PythonRDD.scala:53, PythonRDD[326] at RDD at PythonRDD.scala:53]
Time Random Forest  59.64871859550476
Average Error Random Forest 
0.48753411978612404
Time LogisticRegression  17.92341113090515
Average Error Logistic Regression
0.48753411978612404
Time DecisionTree  2.0962634086608887
Average Error Decision Tree
0.7020941008279739
Time Naive Bayes  5.224005699157715
Average Error Naive Bayes
0.3610948413948317
RF: 
Доля верных предсказаний: 81.353591160221%
Time Random Forest  39.391579151153564
Precision = 0.8470208622233535
Recall = 0.8135359116022098
F1-score = 0.8169610842219016
accuracy: = 0.81353591160221
Precision = 0.8470208622233535
Recall = 0.8135359116022098
F1-score = 0.8169610842219016
accuracy: = 0.81353591160221
LR:
Доля верных предсказаний: 91.76136363636364%
Time 

In [ ]:
#!spark --cluster bigdata-course-spark-cluster-jdrtk8xm --session gr0373-Vlasyuk-Serebryakova